In [7]:
import pandas as pd
import altair as alt
import polars as pl


depara = pl.from_pandas(pd.read_excel('data/repasse/graficos.xlsx', sheet_name='nome_ac'))
depara_repasse = pl.from_pandas(pd.read_excel('data/repasse/graficos.xlsx', sheet_name='depara_repasse'))

depara = depara.join(depara_repasse, left_on='nome_sku', right_on='SKU', how='left')

df = pd.read_csv('precos.csv', sep=';', encoding='latin1')
df.columns = ['ano', 'mes', 'tipo_operação', 'sku', 'nome_sku', 'volume', 'caixas', 'unidades', 'faturamento','cobertura']

df = pl.from_pandas(df).with_columns([
    pl.col(['volume', 'faturamento']).str.strip('-R$ ').str.replace(',','.').cast(pl.Float64),
    pl.when(pl.col('tipo_operação') == 'DIRETA').then(pl.lit('DIRETA')).otherwise(pl.lit('ROTA')).alias('tipo_operação')
]).filter(pl.col('volume') > 0).join(depara, left_on='sku', right_on='sku', how='left').filter(pl.col('ano') == 2023)

df = df.groupby(['mes', 'nome_slide', 'tipo_operação', 'Grupo', 'Caminho']).agg([
    pl.sum('faturamento'),
    pl.sum('volume'),
    pl.sum('caixas'),
    pl.sum('unidades'),
    pl.sum('cobertura').alias('distribuição'),
]).with_columns([
    (pl.col('faturamento')/pl.col('unidades')).alias('TTV'),
    pl.col('nome_slide').alias('nome_sku')
    ])#.filter(pl.col('mes') < 11)

df



mes,nome_slide,tipo_operação,Grupo,Caminho,faturamento,volume,caixas,unidades,distribuição,TTV,nome_sku
i64,str,str,str,str,f64,f64,i64,i64,f64,f64,str
1,"""SLA LT350""","""DIRETA""","""Nenhum""","""data\img\SodaA…",98465.04,182.45,4344,52128,null,1.888909,"""SLA LT350"""
1,"""H2OH P1,5""","""DIRETA""","""Premium""","""data\img\H2OHL…",582531.83,1578.96,17544,105264,null,5.534008,"""H2OH P1,5"""
1,"""SLA P2""","""DIRETA""","""Multi2""","""data\img\SodaA…",215373.33,813.84,6782,40692,null,5.292768,"""SLA P2"""
1,"""GCAD LT350""","""DIRETA""","""Nenhum""","""data\img\Guara…",211864.79,389.84,9282,111384,null,1.902112,"""GCAD LT350"""
1,"""GCA P3""","""DIRETA""","""Multi2""","""data\img\Guara…",76275.87,302.88,2524,10096,null,7.555058,"""GCA P3"""
1,"""GCA P1,5""","""DIRETA""","""Multi1""","""data\img\Guara…",968267.88,3601.71,40019,240114,null,4.032534,"""GCA P1,5"""
1,"""PC P1""","""DIRETA""","""Nenhum""","""data\img\Pepsi…",29966.68,85.26,1421,8526,null,3.514741,"""PC P1"""
1,"""BARE P1""","""DIRETA""","""Multi1""","""data\img\BareG…",105226.89,428.1,7135,42810,null,2.457998,"""BARE P1"""
11,"""H2OH P500""","""DIRETA""","""Premium""","""data\img\H2OHL…",705701.36,1098.06,18301,219612,null,3.213401,"""H2OH P500"""


In [8]:
# chart = alt.Chart(df).mark_bar().encode(
#     y = alt.Y('nome_sku:N', axis=None),
#     x = 'volume:Q'
# ).properties(height=alt.Step(100))

# images = alt.Chart(df).mark_image(width=80, yOffset=-5).encode(
#     y = alt.Y('nome_sku:N', axis = alt.Axis(domainOpacity=0, ticks=False, labelFontSize=14)),
#     url = 'Caminho:N'
# ).properties(height=alt.Step(100))

# alt.concat(images, chart).configure_concat(spacing=0).configure_view(strokeOpacity=0)

In [9]:
source = df.filter(pl.col('Grupo').is_in(['Multi1'])).filter(pl.col('nome_sku') == "SAB P1").to_pandas()

chart = alt.Chart(source).mark_area(interpolate='natural', stroke='lightgrey', fillOpacity=0.8).encode(
    y = alt.Y('TTV:Q', axis=alt.Axis(grid=False)).scale(clamp=True, zero=False),
    x = 'mes:O',
    tooltip=['mes','TTV'],
    color = alt.Color('nome_sku:N', legend=None).scale(scheme='redyellowblue')
)

label = chart.mark_text(yOffset=-7, fontSize=10).encode(
    y = alt.Y('TTV:Q').axis(labels=False, ticks=False),
    x = 'mes:O',
    text=alt.Text("TTV:Q", format="$.2f"),color=alt.Color()
)

img = alt.Chart(source).mark_image(width=50, baseline='middle').encode(
    y=alt.Y('nome_sku', axis=alt.Axis(domainOpacity=0, ticks=False)).axis(labelPadding=20, grid=False, gridOpacity=0, tickOpacity=0, domainOpacity=0) ,
    url = 'Caminho'
).properties(height = alt.Step(120)).configure_view(strokeOpacity=0)

alt.layer(chart, label).properties(width=400, height = 100).facet(row = alt.Row('nome_sku:N').header(labelAngle=0, labelAlign='left'), column=alt.Column('tipo_operação')).properties(bounds='flush').configure_concat(spacing=0).configure_view(strokeOpacity=0)

alt.FacetChart(...)

In [10]:
## making one at a time

grupos = ['Single','Premium', 'Multi1', 'Multi2']
source = df.filter(pl.col('Grupo').is_in(grupos)).filter(pl.col('tipo_operação') == 'DIRETA')
skus = ['LATAS 269', 'SAB LT350', 'GCA LT350', 'GAT P500', 'GCA P1', 'GCA P1,5', 'GCA P2', 'PC P2', 'SUK P2']#sorted(source.select(pl.col('nome_sku').unique()).to_series().to_list())

def make_chart(df, sku, y_var):
    
    ymin = min(df.filter(pl.col('nome_sku') == sku).select(pl.col('TTV')).to_series().to_list())-0.2
    ymax = min(df.filter(pl.col('nome_sku') == sku).select(pl.col('TTV')).to_series().to_list())+0.3

    chart = alt.Chart(df.filter(pl.col('nome_sku') == sku).to_pandas()).mark_area(interpolate='natural', stroke='lightgrey', fillOpacity=0.8,
        color=alt.Gradient(
        gradient='linear',
        stops=[alt.GradientStop(color='white', offset=0),
               alt.GradientStop(color='#0F5C87', offset=1)],
        x1=1,
        x2=1,
        y1=1,
        y2=0
    )).encode(
        y = alt.Y(y_var, axis=alt.Axis(grid=False)).scale(clamp=True, zero=False, domain=[ymin, ymax]),
        x = alt.X('mes:N', axis=alt.Axis(grid=False, title=None, labelFontSize=20)).scale(clamp=True, zero=False),
        tooltip=['mes',y_var]
    ).properties(height=120, width=1600)

    label = chart.mark_text(yOffset=-12, fontSize=20, fontStyle='bold').encode(
        y = alt.Y(y_var).axis(labels=False, ticks=False, title=None),
        x = 'mes:N',
        text=alt.Text(y_var, format="$.2f"),color=alt.Color()
    )

    img = alt.Chart(df.filter(pl.col('nome_sku') == sku).to_pandas()).mark_image(
        width=200,
        yOffset=-20,
        xOffset=60,
        baseline='middle').encode(
        y=alt.Y('nome_sku').axis(
            labelPadding=70,
            #labelOffset=15,
            labelAlign='center',
            labelBaseline='middle',
            grid=False,
            gridOpacity=0,
            tickOpacity=0,
            domainOpacity=0,
            title=None,
            labelFontSize=20,
            labelFontStyle='bold'),
        url = 'Caminho:N'
    ).properties(height=120)
    return alt.hconcat(img, alt.layer(chart, label))

individual_graphs = [
    make_chart(source, sku, 'TTV:Q')
for sku in skus]

chart = alt.vconcat(*individual_graphs).configure_concat(spacing=-10).configure_view(opacity=0)

#chart.save('chart.png', ppi=400, scale_factor=4)

chart


alt.VConcatChart(...)

In [21]:
source_bar = df.with_columns([
    (pl.col('volume')/1000).round(1).alias('new_y')
    ]).to_pandas()

bar = alt.Chart(source_bar).mark_bar(color = '#0F5C87').encode(
    x = alt.X('mes:O').axis(labelFontSize=14, labelAngle=0),
    y = alt.Y('sum(new_y):Q').axis(None) 
)

text = alt.Chart(source_bar).mark_text(yOffset=-10, fontSize=14).encode(
    x = 'mes:O',
    y = 'sum(new_y):Q',
    text = 'sum(new_y):Q'
)

(bar + text).properties(width=800).configure_view(opacity=0)

alt.LayerChart(...)